# Double-check tweet language with TextCat

Source: http://www.let.rug.nl/~vannoord/TextCat/

In [53]:
import os
import pandas as pd
import pipes
import re
import tempfile
from IPython.display import clear_output

In [52]:
DATADIR = "/home/erikt/projects/puregome/data/text/"
IDSTR = "id_str"
TEXT = "text"
TEXTCATDIR = "/home/erikt/software/textcat/"
TEXTCAT = "text_cat"
PERL = "/usr/bin/perl"
TEXTUNKNOWN = "I don't know; Perhaps this is a language I haven't seen before?"
UNKNOWN = "unknown"
OR = " or "
DUTCH = "dutch"
LANG = "lang"
OTHER = "other"

In [54]:
def squeal(text=None):
    clear_output(wait=True)
    if not text is None: print(text)

In [40]:
def createPipe(command):
    pipe  = pipes.Template()
    pipe.append(command,"--")
    return(pipe)

def writeAllPipeInput(pipe,text):
    tmpFile = tempfile.NamedTemporaryFile(delete=False)
    f = pipe.open(tmpFile.name,"w")
    f.write(text)
    f.close()
    return(tmpFile.name)

def readPipeOutput(tmpFileName):
    tmpFile = open(tmpFileName,"r")
    textOut = ""
    for line in tmpFile: textOut += line
    tmpFile.close()
    os.unlink(tmpFileName)
    return(textOut)

def textcatString2List(text):
    if text == TEXTUNKNOWN: return([UNKNOWN])
    else: return(text.split(OR))

def textcat(textIn):
    pipe = createPipe(PERL+" "+TEXTCATDIR+TEXTCAT)
    tmpFileName = writeAllPipeInput(pipe,textIn)
    textOut = readPipeOutput(tmpFileName)
    return(textcatString2List(textOut.strip()))

In [ ]:
FILEPATTERN = "20200701"

files = sorted(os.listdir(DATADIR))
for inFileName in files:
    if re.search(FILEPATTERN,inFileName):
        df = pd.read_csv(DATADIR+inFileName,index_col=IDSTR)
        lang = []
        for i in range(0,len(df)):
            if i % 100 == 0: squeal(str(i)+" "+str(len(df))+" "+inFileName)
            text = re.sub(r"\\n"," ",df.iloc[i][TEXT])
            textcatOut = textcat(text.lower())
            if DUTCH in textcatOut: lang.append(DUTCH)
            elif UNKNOWN in textcatOut: lang.append(UNKNOWN)
            else: lang.append(textcatOut[0])
        df[LANG] = lang
        df.to_csv(inFileName,compression="gzip")

47800 51943 20200701-09.out.gz
